In [2]:
!pip install tflearn

     |████████████████████████████████| 112kB 2.9MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=52741ab669e0fcb42c3913b2182c1f48c5b70e19dd95c96225359c1cba561697
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [3]:
#Imports
import nltk
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

#Loading Data
with open("training.json") as file:
	data = json.load(file)

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
#Initializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

#Looping through our data
for intent in data['intents']:
	for pattern in intent['patterns']:
		pattern = pattern.lower()
    		#Creating a list of words
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent['tag'])

	if intent['tag'] not in labels:
	  labels.append(intent['tag'])

In [8]:
print(labels)

['greeting', 'Konfirmasi_nama', 'Konfirmasi_NIK', 'Konfirmasi_alamat', 'diagnosa', 'Konfirmasi_diagnosa', 'K_anak_apa', 'k_anak_dimana', 'k_anak_Kapan', 'k_anak_siapa', 'k_anak_siapa_yang_melapor']


In [10]:
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1
	training.append(bag)
	output.append(output_row)
#Converting training data into NumPy arrays
training = np.array(training)
output = np.array(output)

#Saving data to disk
with open("data.pickle","wb") as f:
	pickle.dump((words, labels, training, output),f)

In [12]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [14]:
ops.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)
model.save("model.tflearn")

Training Step: 1799  | total loss: 1.56465 | time: 0.074s
| Adam | epoch: 200 | loss: 1.56465 - acc: 0.5597 -- iter: 64/66
Training Step: 1800  | total loss: 1.48619 | time: 0.077s
| Adam | epoch: 200 | loss: 1.48619 - acc: 0.5483 -- iter: 66/66
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
## Coba inferensi

def bag_of_words(s, words):
	bag = [0 for _ in range(len(words))]
	
	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i,w in enumerate(words):
			if w == se:
				bag[i] = 1

	return np.array(bag)

In [21]:
message = "orangtua"
results = model.predict([bag_of_words(message,words)])[0]
print(results)

result_index = np.argmax(results)
print(result_index)
tag = labels[result_index]
print(tag)

[2.2367582e-02 8.3419718e-03 5.9065386e-03 7.3194075e-03 6.9327280e-03
 2.4391701e-02 5.0235790e-06 8.1296303e-06 1.6743914e-04 6.3481623e-01
 2.8974324e-01]
9
k_anak_siapa
